# Machine Learning in Python - Group Project 1

**Due Friday, March 10th by 16.00 pm.**

*include contributors names here (such as Name1, Name2, ...)*

## General Setup

In [1]:
# From the instructions, we can see that the data is taken from this library (but then modified) so we install this
# package and use the data for feature engineering!
#!pip install schrutepy
#!pip install nltk

In [2]:
# Add any additional libraries or submodules below

# Data libraries
import numpy as np
import pandas as pd
from schrutepy import schrutepy
import itertools

# Sentiment Analysis
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# download the vader lexicon 
#nltk.download('vader_lexicon')

"""Hutto, C.J. & Gilbert, E.E. (2014). VADER: A Parsimonious Rule-based Model for
Sentiment Analysis of Social Media Text. Eighth International Conference on
Weblogs and Social Media (ICWSM-14). Ann Arbor, MI, June 2014."""

# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting defaults
plt.rcParams['figure.figsize'] = (8,5)
plt.rcParams['figure.dpi'] = 80

# sklearn modules that are necessary
from sklearn.preprocessing import PowerTransformer

In [ ]:
# First we load the data 
data = pd.read_csv("the_office.csv")
# and our external data from SchrutePy. (Referenced at end)
data_from_schrutepy = schrutepy.load_schrute()

After making sure that all the necessary libraries or submodules are uploaded here, please follow the given skeleton to create your project report. 
- Your completed assignment must follow this structure 
- You should not add or remove any of these sections, if you feel it is necessary you may add extra subsections within each (such as *2.1. Encoding*). 

**Do not forget to remove the instructions for each section in the final document.**

## 1. Introduction

*This section should include a brief introduction to the task and the data (assume this is a report you are delivering to a client).* 

- If you use any additional data sources, you should introduce them here and discuss why they were included.

- Briefly outline the approaches being used and the conclusions that you are able to draw.

In [ ]:
"""FINISH"""

## 2. Exploratory Data Analysis and Feature Engineering

*Include a detailed discussion of the data with a particular emphasis on the features of the data that are relevant for the subsequent modeling.* 

- Including visualizations of the data is strongly encouraged - all code and plots must also be described in the write up. 
- Think carefully about whether each plot needs to be included in your final draft - your report should include figures but they should be as focused and impactful as possible.

*Additionally, this section should also implement and describe any preprocessing / feature engineering of the data.*

- Specifically, this should be any code that you use to generate new columns in the data frame `d`. All of this processing is explicitly meant to occur before we split the data in to training and testing subsets. 
- Processing that will be performed as part of an sklearn pipeline can be mentioned here but should be implemented in the following section.*

**All code and figures should be accompanied by text that provides an overview / context to what is being done or presented.**

### 2.1 Exploratory Data Analysis

First we will explore the given dataset and show what the SchrutePy dataset looks like, which we downloaded from the package SchrutePy (reference below). This was hinted at in the instructions and will give us extra data to make our model.

Below is the given dataset from the_office.csv

In [ ]:
data.head(2)

And here is the external dataset we use from SchrutePy:

In [ ]:
data_from_schrutepy.head(2)

This gives us a quick overview of what our columns are, how the data is inputted and how the overall structure of the data is. This is important for us so we can create efficient code later!
Now lets quickly describe our data:

In [ ]:
display(data.describe().T.round(2))

We can see we have 186 episodes in total, average rating of 8.25 (good show), several other parameters which will be important later when we feature engineer. We can see for example there is quite a large standard deviation in the amount of votes, which could skew our ratings. We also see our rating (target variable) has a standard deviation of 0.54, whereas the mean is 8.25. This shows there is enough spread of rating to make non-uniform predictions.

Are there any missing values?

In [ ]:
print('The DataFrame: data, has ',data.isna().sum().sum(),' NaN values')

What about our external dataset?

In [ ]:
data_from_schrutepy.isna().sum()

We can see we have the columns text and text_w_directions have NaN values. However, we will not be using these anyways since its just the script! So we do not have to worry about it :)

In [ ]:
# We will no get our numerical data so we can visualize it:
num_data = data.copy()
num_data = num_data.drop(columns = ['episode_name', 'director', 'writer', 'air_date', 'main_chars'])

Now lets take a look at the distribution of the columns. (except of course the season number, episode, and air date)

In [ ]:
fig, axes = plt.subplots(figsize=(15,7), ncols=3, nrows=2)
axes = axes.flatten()

#define our columns
lst = ['imdb_rating','total_votes', 'n_lines', 'n_directions','n_words', 'n_speak_char']

for i, ax in enumerate(axes):
    sns.histplot(data = num_data[lst[i]], bins=30, ax=ax)
    ax.set_title(num_data[lst[i]].name)
    ax.set_ylabel("")
    ax.set_xlabel("")

plt.tight_layout()
plt.show()

Seeing the distribution is important because it lets us see several hints such as the Skewness, Gaussian-ness, etc. We can see the IMDB rating follows what seems like a normal distribution, whereas total votes decreasing seemingly exponentially. While the mathematical relation isn't necessarily important for each of the variables, it is important to understand total votes is skewed.

The graphs above allow us to visualise the distributions of all of our numerical features. We can see that some features are skewed and others follow a normal gaussian distribution. From this information, we will implement a box cox transformation to transform all non-gaussian features to gaussian. This will be done in the feature engineering section. Doing so will improve the predictions made by our linear regression model further down the line. 


Now before we move on to visualizing the correlation between variables, we want to see a breakdown of the relationship between writers and directors. After group discussion, we already know we want to include writers and directors in our model, but we need to make encoding of these variables. 

What is important to us is not the directors which had one hit wonders: the directors which had directed one episode and got lucky with a good rating. Maybe they are really good, but we are looking for consistency. The director for our final episode has to be consistently good.

<b> We set a threshold to choose which directors to encode in our model:</b> we want directors to have directed at least ***5 shows*** and then from there select the ***best 5***  

In [ ]:
#get director dataframe which has their ratings and get a new column calculating the amount of times they directed
director = data[['director', 'imdb_rating']].reindex(columns=list(['director', 'imdb_rating', 'freq']), fill_value=1)
#calculates the sum
director = director.groupby('director')[['director', 'imdb_rating', 'freq']].sum() 
director['mean_rating'] = round(director['imdb_rating'] / director['freq'], 2)
# and sort the values for us on frequency (explained below) with a mean rating in the 75% percentile. 
director[director['freq']>=5].drop(columns = 'imdb_rating').sort_values(by='mean_rating', ascending=False).head(6)

We find that using this criteria, we get some of the consistnently best directors. Fortunately for us, we can see that all our directors with high frequency also have fairly good ratings, with Paul Feig having a very high rating consistently!

Now lets do the same with writer. Here some times the writers collab. We will NOT split up the writers individually because we believe collaborations between writers can lead to better results than the writers individually together!

In [ ]:
#get director dataframe which has their ratings and get a new column calculating the amount of times they directed
writer = data[['writer', 'imdb_rating']].reindex(columns=list(['writer', 'imdb_rating', 'freq']), fill_value=1)
#calculates the sum
writer = writer.groupby('writer')[['writer', 'imdb_rating', 'freq']].sum() 
writer['mean_rating'] = round(writer['imdb_rating'] / writer['freq'], 2)
# and sort the values for us on frequency (explained below) with a mean rating in the 75% percentile. 
writer[writer['freq']>=5].drop(columns = 'imdb_rating').sort_values(by='mean_rating', ascending=False).head(6)

Using the same reasoning, we can find the best consistently best 5 writers which we will choose to encode!

Now that we have done all of this: It is time to see the correlation between our variables so we can choose the best selection for our model to reduce feature dependence!

In [ ]:
#heatmap
sns.set(rc={'figure.figsize': (14, 8)})
sns.heatmap(num_data.corr(), annot = True, fmt = '.2f', linewidths = 2)
plt.title("Correlation Heatmap for all ")
plt.show()

We can see some very important things to reduce colinearity/codependence:

* n_lines and n_words is very highly correlated (obviously) so we will deal with that later. Other n_* lines are also slightly correlated so we will find a way to deal with those too.
* imbd_rating and total_votes is also very highly correlated. 
    + We need to be careful here because this implies episodes with a large number of votes often is rated more highly. We cannot have number of votes as an input variable in our model because for a reunion episode, we obviosuly cannot say a priori the amount of votes we will get. To deal with this we will look deeper into the problem: We can adjust the imdb rating based on the amount of votes it got or choose to ignore the correlation if we find the relationship to be insignificant. More on this later...
* The same is with rating and season number. However, since we cannot account for the season number in our reunion episode, we cannot do anything about this either. We could weight each season to imbd rating but do not believe this to be outside the scope of this project. It would be great for further research!

Firstly, lets look at how we can deal with number of ____ feautures

In [ ]:
# we now want to see how the highly correlated values are relates:
sns.pairplot(data[['n_lines','n_directions','n_words', 'n_speak_char']])

We can see a very clear __linear__ relationship between n_lines and n_words which is good for us since it means we can combine or eliminate one of those feautres. The other variables we do not see as strong relationship so we are hesitant to eliminate those. We will see if testing the model on a reduced parameter space will improve performance later, in which we can reduce some of these feautures.

### 2.2 Feature Engineering

<span style="color:red">***FIX/Adjust BELOW***</span>


#### Features we are eliminating (from the_office.csv) and why:
* We will eliminate the column n_words since we found that n_lines and n_words were highly linearly correlated. Doing so will reduce collinearity in our final model. 
* We will remove season number, episode number and air_date because in our opinion, this should not have an affect on the reunion episode. We consider our personal experience and realize most of the office nowadays is watched through streaming services, which are independent of air date. We do however acknowledge that the rating could be dependent on air_time and season numbers, although marginally

#### Features we will engineer:
* We want to see how many times do the main characters talk during the episode and see if it effects the rating.
* We want to OneHotEncode the writers and directors
* OneHotEncode

#### Will we adjust the IMDB rating according to the number of votes and season number to evenly weight the ratings for our model?
As a group, we have decided to _______

#### Finally, we will standardise all of our final numerical features
This includes standardising the following: 

#### 2.2.1 How many times do characters speak?

We can get this data from our external source: data_from_schrutepy

In [ ]:
# make a data which is how many times does {Character} speak in each episode?
speak_count = pd.DataFrame(data_from_schrutepy.groupby(by=['season','episode','character']).count()['text']).unstack()
speak_count.head(1)

We can see that we now have 773 characters in the office, most of which do not speak: a lot of minor characters we do not wish to include in our model. So now lets filter the main characters only: this can be found in the the_office.csv file.

In [ ]:
#correlate this to the main characters in this episode (from the_office.csv) and delete all minor characters
a = data.main_chars.str.split(';')
#this gets all the unique main characters from the_office.csv and save it as main_characters
main_characters = list(itertools.chain.from_iterable(a))
main_characters = pd.DataFrame(main_characters).drop_duplicates().reset_index()[0]

# adjust the dataFrame index
speak_count = speak_count.reset_index(names = ['episode','season', 'character'], col_level=1)
# get just the main character
speak_count_main_characters = speak_count['text'][main_characters].fillna(0)
speak_count_main_characters.head()
#then add the main characters to the model input data!
data[speak_count_main_characters.columns] = speak_count_main_characters
data.head(2)

As we can see, this worked out really well! We now have extra columns which shows how often the main characters speak in the show per episode!

Below we can show a visual representation of how many times each main character speaks!

In [ ]:
sns.boxplot(data[main_characters])
plt.xticks(rotation=45) # rotate x-labels 45 degrees
plt.show()

We can see there are a lot of outliers. This shows some episodes focus on specific characters

    ***REMOVE EPISODES WITH A FOCUS ON ONE CHARACTER*** AS further expansion on project

#### 2.2.2 One Hot Encode the writers and directors:

It is now time to one hot encode the important writers and directors we found in the EDA section.

In [ ]:
# First we define the directors and writers we are looking 
directors = ['Paul Feig','Ken Kwapis','Greg Daniels','Ken Whittingham','Charles McDougall']
writers = ['Greg Daniels','Gene Stupnitsky;Lee Eisenberg','Michael Schur','Paul Lieberstein','B.J. Novak']

# perform encoding on the director column
encoded = pd.get_dummies(data['director'].apply(lambda x: x if x in directors else 'Other_director'))

# merge the encoded dataframe with the original dataframe
data = pd.concat([data, encoded], axis=1)

# drop the original director column
data.drop('director', axis=1, inplace=True)

data.head(1)

In [ ]:
#now lets do the same but with the writers!
# perform encoding on the director column
encoded = pd.get_dummies(data['writer'].apply(lambda x: x if x in writers else 'Other_writer'))

# merge the encoded dataframe with the original dataframe
data = pd.concat([data, encoded], axis=1)

# drop the original director column
data.drop('writer', axis=1, inplace=True)

data.head(1)

#### 2.2.3 Dropping our Unessecary Columns

Its now time to get rid of season, episode,episode_name,total_votes	and air_date so we can start to deal with the numbers of words and votes

In [ ]:
data = data.drop(columns=['season', 'episode', 
                          'episode_name','total_votes',
                         'air_date', 'main_chars'])

#### 2.2.4 What to do with n_lines and n_words

We know from our correlation matrix that these two are highly linearly correlated. That is why we will drop the n_words column (we could also have dropped n_lines instead). 

We will also know create a copy of our dataFrame to start working on the model with

In [ ]:
data = data.drop(columns=['n_words'])

#### 2.2.5 Sentiment Score

We now will run the sentiment analysis on the script per episode. This uses the Vader dictionary to determine how happy/neutral/sad a given script is!

In [ ]:
# instantiate the sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# define some text to analyze (fill the silences (nan) with  ' ')
text = data_from_schrutepy.fillna(' ').groupby(['season', 'episode'])['text'].apply(' '.join)

# analyze the sentiment of the text
scores = [analyzer.polarity_scores(i) for i in text]

In [ ]:
# get the neutral, positive and negative sentiment scores
data['pos_score'] = [scores[i]['pos'] for i in range(len(scores))]
data['neg_score'] = [scores[i]['neg'] for i in range(len(scores))]

### 2.3 Feature Transformation

The final step is to transform our data (using yeo-johnson) to improve our data normality and symmetry! 

In [ ]:
yeo = PowerTransformer()
data_scaled = yeo.fit_transform(data)
data_scaled = pd.DataFrame(data_scaled, columns=data.columns)


In [ ]:
# df will be the dataframe we are working with now
df = data.copy()
df_scaled = data_scaled.copy()

df.to_csv('final_office.csv')
df_scaled.to_csv('final_office_scaled.csv')

## 3. Model Fitting and Tuning

*In this section you should detail your choice of model and describe the process used to refine and fit that model.*

- You are strongly encouraged to explore many different modeling methods (e.g. linear regression, regression trees, lasso, etc.) but you should not include a detailed narrative of all of these attempts. 
- At most this section should mention the methods explored and why they were rejected - most of your effort should go into describing the model you are using and your process for tuning and validatin it.

*For example if you considered a linear regression model, a classification tree, and a lasso model and ultimately settled on the linear regression approach then you should mention that other two approaches were tried but do not include any of the code or any in depth discussion of these models beyond why they were rejected. This section should then detail is the development of the linear regression model in terms of features used, interactions considered, and any additional tuning and validation which ultimately led to your final model.* 

**This section should also include the full implementation of your final model, including all necessary validation. As with figures, any included code must also be addressed in the text of the document.**

In [ ]:
#In GIT there is the final csv file called 
filneame = './final_office.csv'
# in case you guys do not want to run the entire sections above

# to inverse the scaling we used: after prediction do: 
# yeo.inverse_transform(prediction_data)
# and be careful! of the column names!

## 4. Discussion and Conclusions


*In this section you should provide a general overview of **your final model**, its **performance**, and **reliability**.* 

- You should discuss what the implications of your model are in terms of the included features, predictive performance, and anything else you think is relevant.

- This should be written with a target audience of a NBC Universal executive who is with the show and university level mathematics but not necessarily someone who has taken a postgraduate statistical modeling course. 

- Your goal should be to convince this audience that your model is both accurate and useful.

- Finally, you should include concrete recommendations on what NBC Universal should do to make their reunion episode a popular as possible.

**Keep in mind that a negative result, i.e. a model that does not work well predictively, but that is well explained and justified in terms of why it failed will likely receive higher marks than a model with strong predictive performance but with poor or incorrect explanations / justifications.**

In [ ]:
# add that we might want to condiser the timing of the episodes when they aired and that it could be cool to get data to see h
# how much timing afffected the ratings

# add idea about adjusted-IMDB rating based on the amount of votes (takes in the correlation between votes and rating)



## 5. References

*In this section, you should present a list of external sources (except the course materials) that you used during the project, if any*

- Additional data sources can be cited here, in addition to related python documentations, any other webpage sources that you benefited from

In [ ]:
### INSERT reference to shrutepy here

#refernce for trnasformation
I.K. Yeo and R.A. Johnson, “A new family of power transformations to improve normality or symmetry.” Biometrika, 87(4), pp.954-959, (2000).